## Load Packages

In [155]:
from bs4 import BeautifulSoup
import bs4
from urllib.request import Request, urlopen
import requests
import pandas as pd
import re
import numpy as np
import string
import seaborn as sns
import matplotlib.pyplot as plt

!pip install fuzzywuzzy
from fuzzywuzzy import fuzz

## Scrape 247 Recruiting Ranks

In [156]:
rankings = []

# Iterate through rankings of last 5 years
for year in range(2018, 2023):
  print(year)
  for i in range(1, 5):
    req = Request(
        url="https://247sports.com/Season/"+str(year)+"-Basketball/CompositeTeamRankings/?ViewPath=~%2FViews%2FSkyNet%2FInstitutionRanking%2F_SimpleSetForSeason.ascx&Page="+str(i), 
        headers={'User-Agent': 'Mozilla/5.0'}
    )
    webpage = urlopen(req).read()

    soup = BeautifulSoup(webpage, features="lxml")
    teams = soup.find_all("li", class_ = "rankings-page__list-item")

    # For every row on the page, append the team and recruiting points to the rankings dataframe
    for row in teams:
      team = row.find("div", class_="team")
      points = row.find("div", class_="points")
      rankings.append([year, team.get_text().strip(), points.get_text().strip()])

2018
2019
2020
2021
2022


In [193]:
# Create pandas dataframe of recruiting rankings
rankings_df = pd.DataFrame(rankings)
rankings_df.columns = ['Year', 'Team', 'Points']
rankings_df['Points'] = pd.to_numeric(rankings_df['Points'], downcast="float")
rankings_df['Year'] = rankings_df['Year'] + 1
rankings_df

,Year,Team,Points
0,2019,Duke,71.120003
1,2019,Kentucky,70.290001
2,2019,Oregon,69.559998
3,2019,LSU,68.910004
4,2019,Kansas,68.139999
...,...,...,...
862,2023,Rice,0.000000
863,2023,Pennsylvania,0.000000
864,2023,Marshall,0.000000
865,2023,Dartmouth,0.000000


In [158]:
# Aggregate ranking points across years
agg_rankings_df = rankings_df.groupby('Team', as_index=False)['Points'].sum().sort_values('Points', ascending=False)
agg_rankings_df

,Team,Points
57,Duke,350.179993
103,Kentucky,347.869995
99,Kansas,323.239990
147,North Carolina,322.429993
104,LSU,320.880005
...,...,...
234,UMBC,0.000000
238,UNC Wilmington,0.000000
242,UT-Arlington,0.000000
248,Utah Valley,0.000000


## Scrape ESPN Men's Basketball Records

In [194]:
# Iterate through records of last 5 years

teams_list = []
records_list = []
years_list = []

for year in range(2019, 2024):
  print(year)
  req = Request(
      url="https://www.espn.com/mens-college-basketball/standings/_/season/"+str(year), 
      headers={'User-Agent': 'Mozilla/5.0'}
  )
  webpage = urlopen(req).read()

  soup = BeautifulSoup(webpage, features="lxml")


  # Get teams
  teams = soup.find_all(class_="hide-mobile")
  for team in teams:
    teams_list.append(team.get_text())

  # Get records
  tables = soup.find_all(class_="Table__Scroller")
  for table in tables:
    stats = table.find_all(class_="stat-cell")
    for i in range(len(stats)):
      if (i%10 == 3):
        records_list.append(stats[i].get_text())
        years_list.append(year)

2019
2020
2021
2022
2023


In [195]:
records_df = pd.DataFrame({
    'Team': teams_list,
    'Record': records_list,
    'Year': years_list
})
# https://stackoverflow.com/questions/54278595/extract-part-of-a-string-with-regex-before-hyphen-followed-by-digits
records_df['Wins'] = records_df['Record'].str.extract('(.*)-\d+')
records_df['Losses'] = records_df['Record'].str.extract('\d+-(.*)')
records_df['Wins'] = pd.to_numeric(records_df['Wins'], downcast="integer")
records_df['Losses'] = pd.to_numeric(records_df['Losses'], downcast="integer")
records_df['Win Percentage'] = records_df['Wins'] / (records_df['Wins'] + records_df['Losses'])
records_df = records_df[['Team', 'Year', 'Wins', 'Losses', 'Win Percentage']]
records_df

,Team,Year,Wins,Losses,Win Percentage
0,Liberty Flames,2019,29,7,0.805556
1,Lipscomb Bisons,2019,29,8,0.783784
2,North Florida Ospreys,2019,16,17,0.484848
3,Florida Gulf Coast Eagles,2019,14,18,0.437500
4,NJIT Highlanders,2019,22,13,0.628571
...,...,...,...,...,...
1769,Tarleton Texans,2023,7,7,0.500000
1770,UT Rio Grande Valley Vaqueros,2023,8,6,0.571429
1771,California Baptist Lancers,2023,8,7,0.533333
1772,New Mexico State Aggies,2023,7,7,0.500000


In [196]:
# Aggregate team record across years
agg_records_df = records_df.groupby('Team', as_index=False)[['Wins', 'Losses']].sum().sort_values('Wins', ascending=False)
agg_records_df['Win Percentage'] = agg_records_df['Wins'] / (agg_records_df['Wins'] + agg_records_df['Losses'])
agg_records_df

,Team,Wins,Losses,Win Percentage
101,Gonzaga Bulldogs,135.0,14.0,0.906040
113,Houston Cougars,130.0,23.0,0.849673
131,Kansas Jayhawks,122.0,29.0,0.807947
141,Liberty Flames,115.0,32.0,0.782313
72,Duke Blue Devils,113.0,33.0,0.773973
...,...,...,...,...
236,Queens University Royals,11.0,4.0,0.733333
274,Southern Indiana Screaming Eagles,8.0,7.0,0.533333
142,Lindenwood Lions,6.0,9.0,0.400000
286,Stonehill Skyhawks,5.0,11.0,0.312500


## Combine Datasets

### Match Team Names Across Datasets

In [197]:
# Create list of team names for each dataset
team_names_short = agg_rankings_df['Team']
team_names_full = agg_records_df['Team']

In [214]:
# Perform fuzzy string matching
tuples_list = [max([(fuzz.token_set_ratio(i,j),j) for j in team_names_short]) for i in team_names_full]

# Map matches to a list and save as a dataframe
similarity_score, fuzzy_match = map(list,zip(*tuples_list))
matches_df = pd.DataFrame({"Full Name":team_names_full, "Shortened Name": fuzzy_match, "Similarity Score":similarity_score})

# Remove inaccurate matches
potential_matches_df = matches_df.loc[matches_df['Similarity Score'] >= 59]
correct_matches_df = potential_matches_df.loc[~potential_matches_df["Shortened Name"].isin(['NC State', 'Florida', 'Alabama', 'Texas', 'Utah', 'Maryland', 'Arkansas', 'Illinois', 'Washington', 'Houston',
                                                             'Tennessee', 'Southern Miss', 'Michigan', 'North Carolina', 'Kentucky', 'Jacksonville', 'UMass Lowell',
                                                             'North Dakota State', 'Purdue', "St. Mary's", 'Kansas', 'Bellarmine', 'Boston University', 'Idaho', 'San Diego',
                                                             'Tennessee State', 'Louisiana', 'Missouri State', 'Mississippi State', 'California', 'Northwestern',
                                                             'North Dakota', 'Colorado', 'Connecticut', 'Indiana'])][['Full Name', 'Shortened Name']]
correct_matches_df

,Full Name,Shortened Name
101,Gonzaga Bulldogs,Gonzaga
141,Liberty Flames,Liberty
72,Duke Blue Devils,Duke
22,Belmont Bruins,Belmont
20,Baylor Bears,Baylor
...,...,...
35,Cal Poly Mustangs,Cal Poly
58,Columbia Lions,Columbia
282,St. Thomas - Minnesota Tommies,St. Thomas
50,Chicago State Cougars,Chicago State


In [215]:
# Append proper matches manually where necessary
manual_pairings = [
    ["NC State Wolfpack", "NC State"],
    ["Florida Gators", "Florida"],
    ["Alabama Crimson Tide", "Alabama"],
    ["Texas Longhorns", "Texas"],
    ["Utah Utes", "Utah"],
    ["Maryland Terrapins", "Maryland"],
    ["Arkansas Razorbacks", "Arkansas"],
    ["Illinois Fighting Illini", "Illinois"],
    ["Washington Huskies", "Washington"],
    ["Houston Cougars", "Houston"],
    ["Tennessee Volunteers", "Tennessee"],
    ["Southern Miss Golden Eagles", "Southern Miss"],
    ["Michigan Wolverines", "Michigan"],
    ["North Carolina Tar Heels", "North Carolina"],
    ["Kentucky Wildcats", "Kentucky"],
    ["Jacksonville Dolphins", "Jacksonville"],
    ["UMass Lowell River Hawks", "UMass Lowell"],
    ["North Dakota State Bison", "North Dakota State"],
    ["Purdue Boilermakers", "Purdue"],
    ["Saint Mary's Gaels", "St. Mary's"],
    ["Kansas Jayhawks", "Kansas"],
    ["Bellarmine Knights", "Bellarmine"],
    ["Boston University Terriers", "Boston University"],
    ["Idaho Vandals", "Idaho"],
    ["San Diego Toreros", "San Diego"],
    ["Tennessee State Tigers", "Tennessee State"],
    ["Louisiana Ragin' Cajuns", "Louisiana"],
    ["Missouri State Bears", "Missouri State"],
    ["Mississippi State Bulldogs", "Mississippi State"],
    ["California Golden Bears", "California"],
    ["Northwestern Wildcats", "Northwestern"],
    ["North Dakota Fighting Hawks", "North Dakota"],
    ["Colorado Buffaloes", "Colorado"],
    ["South Florida Bulls", "USF",],
    ["UT Rio Grande Valley Vaqueros", "Texas-Rio Grande Valley"],
    ["SE Louisiana Lions", "Southeastern Louisiana"],
    ["Central Michigan Chippewas", "Central Michigan"],
    ["East Tennessee State Buccaneers", "East Tennessee State"],
    ["Eastern Illinois Panthers", "Eastern Illinois"],
    ["Eastern Kentucky Colonels", "Eastern Kentucky"],
    ["Florida International Panthers", "FIU"],
    ["George Washington Colonials", "George Washington"],
    ["Hawai'i Rainbow Warriors", "Hawaii"],
    ["UIC Flames", "Illinois-Chicago"],
    ["Loyola Maryland Greyhounds", "Loyola Maryland"],
    ["UMass Minutemen", "Massachusetts"],
    ["Middle Tennessee Blue Raiders", "Middle Tennessee State"],
    ["Kansas City Roos", "Missouri-Kansas City"],
    ["Mount St. Mary's Mountaineers", "Mount St. Mary's"],
    ["North Texas Mean Green", "North Texas"],
    ["Sam Houston Bearkats", "Sam Houston State"],
    ["UConn Huskies", "Connecticut"],
    ["Indiana Hoosiers", "Indiana"]
]
manual_pairings_df = pd.DataFrame(manual_pairings)
manual_pairings_df.columns = ['Full Name', 'Shortened Name']

In [216]:
# Concatenate correct matches and manual pairings
name_pairings_key=pd.concat([correct_matches_df, manual_pairings_df])

# Merge datasets using name pairings as a key
rankings_versus_wins = records_df.merge(name_pairings_key, left_on='Team', right_on='Full Name').merge(rankings_df, left_on=['Shortened Name', 'Year'], right_on=['Team', 'Year'])[['Shortened Name', 'Year', 'Wins', 'Win Percentage', 'Points']]
agg_rankings_versus_wins = agg_records_df.merge(name_pairings_key, left_on='Team', right_on='Full Name').merge(agg_rankings_df, left_on='Shortened Name', right_on='Team')[['Shortened Name', 'Points', 'Wins']]

In [ ]:
rankings_versus_wins.head()

In [ ]:
agg_rankings_versus_wins.head()

## Export dataframes

In [220]:
# Export rankings versus wins by team by year
rankings_versus_wins.to_csv("Wins versus Recruiting Ranking.csv")

# Export rankings versus wins by team aggregated across all years
agg_rankings_versus_wins.to_csv("Agg Wins versus Recruiting Ranking.csv")